In [3]:
import os
import numpy as np
import pandas as pd
from pathlib import PosixPath
from torch.utils.data import Dataset, DataLoader
from fastai.tabular import *

# GO TO CHECK POINT

# 1. Cohort dataset

In [2]:
cohort = pd.read_csv('/data/knee_mri3/demoInfo/enrollee01.txt', header = 1, 
                     low_memory=False, sep='\t')

In [3]:
cohort.shape

(43164, 26)

In [4]:
cohort.columns

Index(['collection_id', 'enrollee01_id', 'dataset_id',
       'The NDAR Global Unique Identifier (GUID) for research subject',
       'Subject ID how it's defined in lab/project',
       'Date on which the interview/genetic test/sampling/imaging/biospecimen was completed. MM/DD/YYYY',
       'Age in months at the time of the interview/test/sampling/imaging.',
       'Sex of the subject', 'Age in years', 'Ethnicity of participant',
       'Visit name', 'Subcohort assignment', 'Participant in Image Group A',
       'Participant in Image Group B', 'Participant in Image Group C',
       'Participant in Image Group D', 'Participant in Image Group E',
       'Participant in Image Group F', 'Participant in Image Group G',
       'Race of study subject',
       'Participant in first half of cohort, or second half of cohort',
       'Version/code of assessment', 'Site',
       'Indicator variable for having had an interim 6-month visit (18- or 30-month visit)',
       'collection_title', 'promo

In [5]:
cohort_small = cohort.loc[cohort['Visit name'] == 'V00', 
                         [ "Subject ID how it's defined in lab/project",
                          'Sex of the subject',
                         'Age in months at the time of the interview/test/sampling/imaging.',
                         'Ethnicity of participant',
                         'Race of study subject']
                         ]

In [6]:
cohort_small.columns = ['ID', 'sex', 'age_months', 'ethnicity', 'race']

In [7]:
cohort_small.head()

,ID,sex,age_months,ethnicity,race
9592,9370237,M,636.0,Not Hispanic or Latino,White
9593,9370366,F,600.0,Not Hispanic or Latino,White
9594,9371094,M,912.0,Not Hispanic or Latino,White
9595,9371267,F,912.0,Not Hispanic or Latino,White
9596,9371355,M,804.0,Not Hispanic or Latino,White


In [8]:
cohort_small.shape

(4796, 5)

In [9]:
cohort_small.to_pickle('/data/knee_mri3/labels/cohort_small.pickle')

In [10]:
cohort_small = pd.read_pickle('/data/knee_mri3/labels/cohort_small.pickle')

In [9]:
cohort_small.loc[cohort_small.ID == 9176485]

,ID,sex,age_months,ethnicity,race
9890,9176485,M,900.0,Not Hispanic or Latino,White


In [10]:
900 / 12

75.0

# 2. Clinical dataset 

## 2.1 Relevant clinical, function, symptomatic variables at baseline 

In [89]:
predictors = pd.read_excel('data/OAI_TDA_DataDictionary.xlsx')

In [71]:
predictors.head()

,row,variable name,category,oai visit,oai name,oai dataset,unit,data type,specific meaning,comments
0,NaN,id,demographic,NaN,ID,AllClinical,NaN,NaN,NaN,NaN
1,NaN,v00age,demographic,V00,AGE,AllClinical,years,numeric,participant age in years at visit,NaN
2,NaN,v01age,demographic,V01,AGE,AllClinical,years,numeric,participant age in years at visit,NaN
3,NaN,v02age,demographic,V02,AGE,AllClinical,years,numeric,participant age in years at visit,NaN
4,NaN,v03age,demographic,V03,AGE,AllClinical,years,numeric,participant age in years at visit,NaN


In [72]:
predictors = predictors.loc[predictors['oai visit'].isin(['V00', 'P01', 'P02'])]

In [73]:
predictors = predictors.loc[predictors['oai dataset'].isin(['AllClinical', 'Enrollees'])]

In [74]:
predictors.columns

Index(['row', 'variable name', 'category ', 'oai visit', 'oai name',
       'oai dataset', 'unit ', 'data type ', 'specific meaning', 'comments'],
      dtype='object')

In [75]:
predictors = predictors.dropna(subset=['data type '])

In [76]:
predictors['keys'] = predictors['oai visit'] + predictors['oai name']

In [77]:
predictors = predictors.loc[:, ['keys', 'category ', 'specific meaning', 'comments']]

In [78]:
predictors.shape

(770, 4)

In [79]:
predictors.head(20)

,keys,category,specific meaning,comments
1,V00AGE,demographic,participant age in years at visit,NaN
13,P02SEX,demographic,participant sex,1=male; 2=female
14,P02RACE,demographic,racial background,Int 0:3
15,P02HISP,demographic,hispanic or latino background,NaN
16,V00EDCV,subject characteristics,highest grade of school completed,Int 0:5
17,V00MARITST,subject characteristics,marital status,Int 1:5
21,V00LIVENO,subject characteristics,how many people in household,Int 0:6
25,V00LIVE1,subject characteristics,live with spouse,NaN
29,V00LIVE2,subject characteristics,live with romantic partner,NaN
33,V00LIVE3,subject characteristics,live with children,NaN


In [80]:
left = predictors.loc[np.logical_or(predictors['specific meaning'].str.contains('left knee'),
                            np.logical_or(predictors['specific meaning'].str.contains('left hip'),
                                          np.logical_or(predictors['specific meaning'].str.contains('left ankle'),
                                                       predictors['specific meaning'].str.contains('left leg'))
                                         )), :]

In [81]:
left.head()

,keys,category,specific meaning,comments
132,P02KPNLCV,knee symptoms,"left knee pain, aching or stiffness on most da...",NaN
151,P01KPNLEV,knee symptoms,"left knee pain, aching or stiffness: ever had ...",NaN
152,P01KPNLEVY,knee symptoms,"how long ago was the start of left knee pain, ...",NaN
153,P01KPNL12,knee symptoms,"any left knee pain, aching or stiffness in pas...",NaN
154,P01KPL12CV,knee symptoms,"left knee pain, aching or stiffness on most da...",NaN


In [82]:
right = predictors.loc[np.logical_or(predictors['specific meaning'].str.contains('right knee'),
                            np.logical_or(predictors['specific meaning'].str.contains('right hip'),
                                          np.logical_or(predictors['specific meaning'].str.contains('right ankle'),
                                                       predictors['specific meaning'].str.contains('right leg'))
                                         )), :]

In [83]:
predictors.loc[predictors['category '] == 'knee pain']

,keys,category,specific meaning,comments
275,V00KOOSKPR,knee pain,KOOS right knee pain score,NaN
276,V00KPRKN1,knee pain,KOOS right knee pain during twisting/pivoting ...,NaN
277,V00KPRKN2,knee pain,KOOS right knee pain during straightening knee...,NaN
278,V00KPRKN3,knee pain,KOOS right knee pain during bending knee fully...,NaN
279,V00WPRKN1,knee pain,KOOS/WOMAC right knee pain during walking in l...,NaN
280,V00WPRKN2,knee pain,KOOS/WOMAC right knee pain going up or down st...,NaN
281,V00WPRKN3,knee pain,KOOS/WOMAC right knee pain while in bed in las...,NaN
282,V00WPRKN4,knee pain,KOOS/WOMAC right knee pain sitting or lying in...,NaN
283,V00WPRKN5,knee pain,KOOS/WOMAC right knee pain standing upright in...,NaN
284,V00P7RKFR,knee pain,KOOS right knee how often have pain,NaN


In [59]:
predictors['category '].value_counts()

medical history            108
nutrition                  106
strength measures           85
medication                  57
knee function               56
performance measures        56
baseline risk factors       56
physical activity           38
knee symptoms               35
other joint pain            26
knee exam                   24
knee pain                   24
demographic history         18
hip pain                    16
global function             14
back pain                   12
hand exam                   12
subject characteristics     11
demographic                  9
health care access           3
physical exam                3
med                          1
Name: category , dtype: int64

In [84]:
predictors.loc[np.logical_and(predictors['category '] == 'medical history',
                             predictors['specific meaning'].str.contains('knee pain'))]

,keys,category,specific meaning,comments


In [85]:
pain_names = predictors.loc[predictors['specific meaning'].str.contains('knee pain')]

In [86]:
pain_names

,keys,category,specific meaning,comments
131,P02KPNRCV,knee symptoms,"right knee pain, aching or stiffness on most d...",NaN
132,P02KPNLCV,knee symptoms,"left knee pain, aching or stiffness on most da...",NaN
141,P01KPACDCV,knee symptoms,"days of limited activities due to knee pain, a...",NaN
142,P01KPA30CV,knee symptoms,avoided or changed activities to reduce knee p...,NaN
143,P01KPNREV,knee symptoms,"right knee pain, aching or stiffness: ever had...",NaN
144,P01KPNREVY,knee symptoms,"how long ago was the start of right knee pain,...","Int 1,2,5"
145,P01KPNR12,knee symptoms,"any right knee pain, aching or stiffness in pa...",NaN
146,P01KPR12CV,knee symptoms,"right knee pain, aching or stiffness on most d...",NaN
147,P01KPNR12M,knee symptoms,"how many months with right knee pain, aching o...",NaN
148,P01RKP30CV,knee symptoms,"any right knee pain, aching or stiffness in pa...",NaN


In [87]:
predictors.to_pickle('/data/knee_mri3/JL/predict_pain/data/oai_selected_labels.pickle')

## 2.2 Filter them in AllClinical00 dataset

In [90]:
var_names = predictors['oai visit'] + predictors['oai name']

In [91]:
label_dir = PosixPath('/data/knee_mri3/demoInfo/all_clinical/')

In [92]:
ac = pd.read_csv(label_dir/'AllClinical00.txt', sep='|')

In [93]:
var_names = [v for v in var_names if v in ac.columns]

In [94]:
ac = ac.loc[:, ['ID'] + var_names]

In [95]:
ac.shape

(4796, 733)

In [67]:
ac.head()

,ID,V00AGE,V00EDCV,V00MARITST,V00LIVENO,V00LIVE1,V00LIVE2,V00LIVE3,V00LIVE4,V00LIVE5,...,V00VIT4,V00VIT5,V00VIT8,V00VIT9,V00VIT10,V00VIT11,V00VIT12,V00VIT6,V00VIT13,V00VIT7
0,9000099,59,5: Graduate degree,1: Married,2: Two,1: Yes,.: Missing Form/Incomplete Workbook,1: Yes,.: Missing Form/Incomplete Workbook,.: Missing Form/Incomplete Workbook,...,0: No,0: No,0: No,0: No,0: No,0: No,0: No,0: No,0: No,0: No
1,9000296,69,2: Some college,1: Married,1: One,1: Yes,.: Missing Form/Incomplete Workbook,.: Missing Form/Incomplete Workbook,.: Missing Form/Incomplete Workbook,.: Missing Form/Incomplete Workbook,...,0: No,0: No,0: No,0: No,0: No,0: No,0: No,0: No,0: No,0: No
2,9000622,71,1: High school graduate,1: Married,1: One,1: Yes,.: Missing Form/Incomplete Workbook,.: Missing Form/Incomplete Workbook,.: Missing Form/Incomplete Workbook,.: Missing Form/Incomplete Workbook,...,0: No,0: No,0: No,1: Yes,0: No,0: No,0: No,0: No,1: Yes,0: No
3,9000798,56,2: Some college,1: Married,1: One,1: Yes,.: Missing Form/Incomplete Workbook,.: Missing Form/Incomplete Workbook,.: Missing Form/Incomplete Workbook,.: Missing Form/Incomplete Workbook,...,0: No,0: No,0: No,0: No,0: No,0: No,0: No,1: Yes,0: No,0: No
4,9001104,72,2: Some college,2: Widowed,0: Zero,.: Missing Form/Incomplete Workbook,.: Missing Form/Incomplete Workbook,.: Missing Form/Incomplete Workbook,.: Missing Form/Incomplete Workbook,.: Missing Form/Incomplete Workbook,...,0: No,0: No,0: No,1: Yes,1: Yes,0: No,0: No,0: No,1: Yes,0: No


In [68]:
for n, t in zip(ac.columns, ac.dtypes):
    if t == 'O':
#         ac[n] = [e.split(':')[0] for e in ac[n]]
        ac[n] = ac[n].replace({'.: Missing Form/Incomplete Workbook': np.nan})
        ac[n] = ac[n].astype('O')

In [69]:
ac.head()

,ID,V00AGE,V00EDCV,V00MARITST,V00LIVENO,V00LIVE1,V00LIVE2,V00LIVE3,V00LIVE4,V00LIVE5,...,V00VIT4,V00VIT5,V00VIT8,V00VIT9,V00VIT10,V00VIT11,V00VIT12,V00VIT6,V00VIT13,V00VIT7
0,9000099,59,5: Graduate degree,1: Married,2: Two,1: Yes,NaN,1: Yes,NaN,NaN,...,0: No,0: No,0: No,0: No,0: No,0: No,0: No,0: No,0: No,0: No
1,9000296,69,2: Some college,1: Married,1: One,1: Yes,NaN,NaN,NaN,NaN,...,0: No,0: No,0: No,0: No,0: No,0: No,0: No,0: No,0: No,0: No
2,9000622,71,1: High school graduate,1: Married,1: One,1: Yes,NaN,NaN,NaN,NaN,...,0: No,0: No,0: No,1: Yes,0: No,0: No,0: No,0: No,1: Yes,0: No
3,9000798,56,2: Some college,1: Married,1: One,1: Yes,NaN,NaN,NaN,NaN,...,0: No,0: No,0: No,0: No,0: No,0: No,0: No,1: Yes,0: No,0: No
4,9001104,72,2: Some college,2: Widowed,0: Zero,NaN,NaN,NaN,NaN,NaN,...,0: No,0: No,0: No,1: Yes,1: Yes,0: No,0: No,0: No,1: Yes,0: No


In [70]:
ac.to_pickle('/data/knee_mri3/JL/predict_pain/data/AllClinical00_small.pickle')

In [96]:
ac = pd.read_pickle('/data/knee_mri3/JL/predict_pain/data/AllClinical00_small.pickle')

In [97]:
ac.iloc[:10, :10]

,ID,V00AGE,V00EDCV,V00MARITST,V00LIVENO,V00LIVE1,V00LIVE2,V00LIVE3,V00LIVE4,V00LIVE5
0,9000099,59,5: Graduate degree,1: Married,2: Two,1: Yes,NaN,1: Yes,NaN,NaN
1,9000296,69,2: Some college,1: Married,1: One,1: Yes,NaN,NaN,NaN,NaN
2,9000622,71,1: High school graduate,1: Married,1: One,1: Yes,NaN,NaN,NaN,NaN
3,9000798,56,2: Some college,1: Married,1: One,1: Yes,NaN,NaN,NaN,NaN
4,9001104,72,2: Some college,2: Widowed,0: Zero,NaN,NaN,NaN,NaN,NaN
5,9001400,75,4: Some graduate school,2: Widowed,0: Zero,NaN,NaN,NaN,NaN,NaN
6,9001695,52,5: Graduate degree,1: Married,2: Two,1: Yes,NaN,1: Yes,NaN,NaN
7,9001897,72,4: Some graduate school,1: Married,1: One,1: Yes,NaN,NaN,NaN,NaN
8,9002116,61,2: Some college,3: Divorced,0: Zero,NaN,NaN,NaN,NaN,NaN
9,9002316,76,5: Graduate degree,2: Widowed,0: Zero,NaN,NaN,NaN,NaN,NaN


# 3. Combine allClinical and cohort

In [100]:
cohort_small.head()

,index,ID,sex,age_months,ethnicity,race
0,9592,9370237,M,636.0,Not Hispanic or Latino,White
1,9593,9370366,F,600.0,Not Hispanic or Latino,White
2,9594,9371094,M,912.0,Not Hispanic or Latino,White
3,9595,9371267,F,912.0,Not Hispanic or Latino,White
4,9596,9371355,M,804.0,Not Hispanic or Latino,White


In [99]:
cohort_small = cohort_small.reset_index()

In [101]:
pred_df = pd.merge(left=cohort_small, 
               right=ac,
               how='left',
               on='ID')

In [73]:
pred_df.shape

(4796, 734)

In [104]:
pred_df = pred_df.drop('index', 1)

In [105]:
pred_df.iloc[:10, :10]

,ID,sex,age_months,ethnicity,race,V00AGE,V00EDCV,V00MARITST,V00LIVENO,V00LIVE1
0,9370237,M,636.0,Not Hispanic or Latino,White,53,3: College graduate,1: Married,1: One,1: Yes
1,9370366,F,600.0,Not Hispanic or Latino,White,50,2: Some college,1: Married,1: One,1: Yes
2,9371094,M,912.0,Not Hispanic or Latino,White,76,5: Graduate degree,1: Married,1: One,1: Yes
3,9371267,F,912.0,Not Hispanic or Latino,White,76,2: Some college,2: Widowed,1: One,NaN
4,9371355,M,804.0,Not Hispanic or Latino,White,67,2: Some college,1: Married,1: One,1: Yes
5,9372194,F,936.0,Not Hispanic or Latino,White,78,3: College graduate,3: Divorced,0: Zero,NaN
6,9372219,F,612.0,Not Hispanic or Latino,White,51,4: Some graduate school,2: Widowed,0: Zero,NaN
7,9372390,F,624.0,Not Hispanic or Latino,White,52,3: College graduate,1: Married,3: Three,1: Yes
8,9372474,F,852.0,Not Hispanic or Latino,Black or African American,71,5: Graduate degree,1: Married,1: One,1: Yes
9,9372977,F,NaN,Not Hispanic or Latino,White,56,2: Some college,3: Divorced,0: Zero,NaN


# 5. Load FPCA scores 

In [106]:
eta_scores = pd.read_csv('/data/knee_mri3/JL/pain_trajectory_clustering/data/f_scores_2.csv')

In [107]:
eta_scores = eta_scores.iloc[:, 1:]

In [108]:
eta_scores.head()

,combined_id,V1,V2,V3,V4
0,9000099_LEFT,11.184261,-1.695476,-0.338173,0.835294
1,9000296_LEFT,-5.014966,-0.059305,0.079088,0.159873
2,9000296_RIGHT,-4.335862,0.390152,-0.175779,0.229023
3,9000622_LEFT,-2.017891,-0.356249,0.196591,0.129263
4,9000798_RIGHT,-5.327715,-0.223267,0.094702,0.081452


In [18]:
eta_scores.tail()

,combined_id,V1,V2,V3,V4
4074,9999510_RIGHT,-1.206465,0.258790,1.056888,0.501807
4075,9999862_LEFT,-4.549688,0.103468,0.278241,-0.076329
4076,9999862_RIGHT,-4.844343,0.067763,0.109085,-0.071602
4077,9999865_LEFT,1.214970,-1.186112,2.288722,-0.400837
4078,9999865_RIGHT,-4.191619,-0.914949,-0.056639,-0.044809


In [109]:
eta_scores.columns = ['ID_side', 'eta_0', 'eta_1', 'eta_2', 'eta_3']

In [110]:
eta_scores.head()

,ID_side,eta_0,eta_1,eta_2,eta_3
0,9000099_LEFT,11.184261,-1.695476,-0.338173,0.835294
1,9000296_LEFT,-5.014966,-0.059305,0.079088,0.159873
2,9000296_RIGHT,-4.335862,0.390152,-0.175779,0.229023
3,9000622_LEFT,-2.017891,-0.356249,0.196591,0.129263
4,9000798_RIGHT,-5.327715,-0.223267,0.094702,0.081452


In [36]:
eta_mean = eta_scores.loc[:, ['eta_0', 'eta_1', 'eta_2', 'eta_3']].apply(np.mean)
eta_std = eta_scores.loc[:, ['eta_0', 'eta_1', 'eta_2', 'eta_3']].apply(np.std)

eta_scores['std_eta_0'] = (eta_scores['eta_0'] - eta_scores['eta_0'].mean()) / eta_scores['eta_0'].std()
eta_scores['std_eta_1'] = (eta_scores['eta_1'] - eta_scores['eta_1'].mean()) / eta_scores['eta_1'].std()
eta_scores['std_eta_2'] = (eta_scores['eta_2'] - eta_scores['eta_2'].mean()) / eta_scores['eta_2'].std()
eta_scores['std_eta_3'] = (eta_scores['eta_3'] - eta_scores['eta_3'].mean()) / eta_scores['eta_3'].std()

In [37]:
eta_scores.head()

,ID_side,eta_0,eta_1,eta_2,eta_3,std_eta_0,std_eta_1,std_eta_2,std_eta_3
0,9000099_LEFT,11.184261,-1.695476,-0.338173,0.835294,1.677402,-0.996826,-0.331102,1.429741
1,9000296_LEFT,-5.014966,-0.059305,0.079088,0.159873,-0.789935,-0.022829,0.082186,0.278651
2,9000296_RIGHT,-4.335862,0.390152,-0.175779,0.229023,-0.686499,0.244728,-0.170255,0.396500
3,9000622_LEFT,-2.017891,-0.356249,0.196591,0.129263,-0.333444,-0.199597,0.198569,0.226484
4,9000798_RIGHT,-5.327715,-0.223267,0.094702,0.081452,-0.837570,-0.120434,0.097651,0.145002


In [70]:
eta_scores.describe()

,eta_0,eta_1,eta_2,eta_3,std_eta_0,std_eta_1,std_eta_2,std_eta_3
count,4079.000000,4079.000000,4079.000000,4079.000000,4.079000e+03,4.079000e+03,4.079000e+03,4.079000e+03
mean,0.171327,-0.020955,-0.003888,-0.003630,-9.632457e-17,-7.049467e-18,-7.797963e-18,2.131511e-17
std,6.565469,1.679853,1.009614,0.586766,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
min,-5.368141,-12.222689,-7.175451,-3.560293,-8.437278e-01,-7.263571e+00,-7.103269e+00,-6.061465e+00
25%,-4.320304,-0.530117,-0.386930,-0.204506,-6.841295e-01,-3.030990e-01,-3.793948e-01,-3.423435e-01
50%,-2.249909,-0.121479,0.061959,0.040602,-3.687835e-01,-5.984084e-02,6.521950e-02,7.538315e-02
75%,2.256998,0.559492,0.274174,0.175520,3.176727e-01,3.455345e-01,2.754131e-01,3.053177e-01
max,51.734381,11.519074,6.118943,3.156959,7.853674e+00,6.869664e+00,6.064523e+00,5.386453e+00


# 6. Combine posterior probability labels and predictors

In [9]:
labels_df = pd.read_pickle('data/traj_labels_df.pickle')

In [3]:
labels_df.set.value_counts()

0.0    2665
2.0     812
1.0     600
Name: set, dtype: int64

In [5]:
812 / (2665 + 812 + 600)

0.19916605347068922

In [83]:
labels_df.columns

Index(['ids', 'side', 'dess_path', 'pain_meds', 'narcot', 'womac', 'koos',
       'Inferred_KLG', 'oa_status', 'ID', 'pp_0', 'pp_1', 'pp_2',
       'most_likely', 'set'],
      dtype='object')

In [84]:
labels_df.columns = ['ID', 'side', 'dess_path', 'pain_meds', 'narcot', 'womac', 'koos',
       'Inferred_KLG', 'oa_status', 'ID_side', 'pp_0', 'pp_1', 'pp_2',
       'most_likely', 'set']

In [85]:
labels_df.head()

,ID,side,dess_path,pain_meds,narcot,womac,koos,Inferred_KLG,oa_status,ID_side,pp_0,pp_1,pp_2,most_likely,set
0,9477175,LEFT,9477175_LEFT_0.mat,0,0,0.0,100.0,1,0,9477175_LEFT,0.999999,5.458956e-07,6.750792e-08,0,2
1,9794339,LEFT,9794339_LEFT_0.mat,0,0,0.0,97.2,1,0,9794339_LEFT,0.999981,1.873607e-05,7.424682e-07,0,0
2,9282182,RIGHT,9282182_RIGHT_0.mat,0,0,0.0,100.0,0,0,9282182_RIGHT,0.999759,2.402764e-04,3.842015e-07,0,0
3,9359202,LEFT,9359202_LEFT_0.mat,0,0,0.0,100.0,3,1,9359202_LEFT,0.999816,1.823857e-04,1.635602e-06,0,1
4,9754338,RIGHT,9754338_RIGHT_0.mat,0,0,0.0,100.0,1,0,9754338_RIGHT,0.999820,1.789554e-04,8.681482e-07,0,2


In [86]:
labels_df = pd.merge(left=labels_df, 
             right=pred_df,
              how='left',
              on='ID')

In [87]:
labels_df = pd.merge(left = labels_df, 
              right = eta_scores,
              how='right',
              on=['ID_side'])

In [88]:
labels_df.head()

,ID,side,dess_path,pain_meds,narcot,womac,koos,Inferred_KLG,oa_status,ID_side,...,V00VIT10,V00VIT11,V00VIT12,V00VIT6,V00VIT13,V00VIT7,eta_0,eta_1,eta_2,eta_3
0,9477175.0,LEFT,9477175_LEFT_0.mat,0,0,0.0,100.0,1,0,9477175_LEFT,...,0: No,0: No,0: No,1: Yes,1: Yes,0: No,-5.053880,0.005395,0.030075,0.127305
1,9794339.0,LEFT,9794339_LEFT_0.mat,0,0,0.0,97.2,1,0,9794339_LEFT,...,0: No,0: No,0: No,0: No,1: Yes,0: No,-2.059586,-0.306490,0.064717,-0.017907
2,9282182.0,RIGHT,9282182_RIGHT_0.mat,0,0,0.0,100.0,0,0,9282182_RIGHT,...,1: Yes,0: No,0: No,0: No,0: No,1: Yes,-1.092739,-0.772949,0.426054,-0.045561
3,9359202.0,LEFT,9359202_LEFT_0.mat,0,0,0.0,100.0,3,1,9359202_LEFT,...,1: Yes,0: No,0: No,1: Yes,0: No,1: Yes,0.971453,0.365539,0.921799,0.319177
4,9754338.0,RIGHT,9754338_RIGHT_0.mat,0,0,0.0,100.0,1,0,9754338_RIGHT,...,0: No,0: No,0: No,0: No,0: No,0: No,1.309819,-3.678719,-1.595404,0.126684


In [89]:
labels_df.shape

(4079, 752)

In [94]:
labels_df = labels_df.dropna(subset=['eta_0', 'eta_1', 'eta_2', 'eta_3', 'dess_path'])

In [95]:
labels_df.shape

(4077, 752)

In [96]:
with open('data/posterior_probabilities_wide_df.pickle', 'wb') as f:
    pickle.dump([labels_df, eta_mean, eta_std], f)

In [97]:
pd.DataFrame(eta_mean).to_csv('data/eta_mean.csv')

In [98]:
pd.DataFrame(eta_std).to_csv('data/eta_std.csv')

In [99]:
labels_df.to_csv('data/standardized_eta_wide_df.csv')

# Checkpoint

# 7 Make data bunch object - standardization and imputation

In [80]:
# with open('data/posterior_probabilities_wide_df.pickle', 'rb') as f:
#     labels_df, eta_mean, eta_std = pickle.load(f)

In [111]:
labels_df = pd.read_csv('data/standardized_eta_wide_df.csv', low_memory=False)

In [112]:
labels_df.loc[:, ['eta_0', 'eta_1', 'eta_2', 'eta_3']].describe()

,eta_0,eta_1,eta_2,eta_3
count,4077.000000,4077.000000,4077.000000,4077.000000
mean,0.166389,-0.020825,-0.003408,-0.003697
std,6.558821,1.680246,1.009422,0.586829
min,-5.368141,-12.222689,-7.175451,-3.560293
25%,-4.320991,-0.529630,-0.386317,-0.204464
50%,-2.250943,-0.121479,0.061959,0.040602
75%,2.256715,0.561046,0.274333,0.175475
max,51.734381,11.519074,6.118943,3.156959


In [113]:
labels_df = labels_df.iloc[:, 1:]
labels_df.head()

,ID,side,dess_path,pain_meds,narcot,womac,koos,Inferred_KLG,oa_status,ID_side,...,V00VIT10,V00VIT11,V00VIT12,V00VIT6,V00VIT13,V00VIT7,eta_0,eta_1,eta_2,eta_3
0,9477175.0,LEFT,9477175_LEFT_0.mat,0,0.0,0.0,100.0,1.0,0.0,9477175_LEFT,...,0: No,0: No,0: No,1: Yes,1: Yes,0: No,-5.053880,0.005395,0.030075,0.127305
1,9794339.0,LEFT,9794339_LEFT_0.mat,0,0.0,0.0,97.2,1.0,0.0,9794339_LEFT,...,0: No,0: No,0: No,0: No,1: Yes,0: No,-2.059586,-0.306490,0.064717,-0.017907
2,9282182.0,RIGHT,9282182_RIGHT_0.mat,0,0.0,0.0,100.0,0.0,0.0,9282182_RIGHT,...,1: Yes,0: No,0: No,0: No,0: No,1: Yes,-1.092739,-0.772949,0.426054,-0.045561
3,9359202.0,LEFT,9359202_LEFT_0.mat,0,0.0,0.0,100.0,3.0,1.0,9359202_LEFT,...,1: Yes,0: No,0: No,1: Yes,0: No,1: Yes,0.971453,0.365539,0.921799,0.319177
4,9754338.0,RIGHT,9754338_RIGHT_0.mat,0,0.0,0.0,100.0,1.0,0.0,9754338_RIGHT,...,0: No,0: No,0: No,0: No,0: No,0: No,1.309819,-3.678719,-1.595404,0.126684


In [114]:
tmp = labels_df.copy()

for l in left['keys']:
#     assert l in tmp.columns, "{} not in labels_df".format(l)
    if l in labels_df.columns:
        c_name = l + '_C'
        i_name = l + '_I'
        tmp[c_name] = np.nan
        tmp[i_name] = np.nan
        tmp.loc[tmp.side == 'LEFT', i_name] = tmp.loc[tmp.side == 'LEFT', l]
        tmp.loc[tmp.side == 'RIGHT', c_name] = tmp.loc[tmp.side == 'RIGHT', l]

In [115]:
for l in right['keys']:
#     assert l in tmp.columns, "{} not in labels_df".format(l)
    if l in labels_df.columns:
        c_name = l + '_C'
        i_name = l + '_I'
        tmp[c_name] = np.nan
        tmp[i_name] = np.nan
        tmp.loc[tmp.side == 'LEFT', c_name] = tmp.loc[tmp.side == 'LEFT', l]
        tmp.loc[tmp.side == 'RIGHT', i_name] = tmp.loc[tmp.side == 'RIGHT', l]

In [25]:
col_to_drop = list(left['keys']) + list(right['keys'])
col_to_drop = [c for c in col_to_drop if c in labels_df.columns]

In [244]:
col_to_drop

['P02KPNLCV',
 'P01KPNLEV',
 'P01KPNLEVY',
 'P01KPNL12',
 'P01KPL12CV',
 'P01KPNL12M',
 'P01LKP30CV',
 'P01KPL30CV',
 'P01PMLKRCV',
 'V00KOOSKPL',
 'V00KPLKN1',
 'V00KPLKN2',
 'V00KPLKN3',
 'V00WPLKN1',
 'V00WPLKN2',
 'V00WPLKN3',
 'V00WPLKN4',
 'V00WPLKN5',
 'V00P7LKFR',
 'V00P7LKRCV',
 'V00WOMKPL',
 'V00KOOSYML',
 'V00KSXLKN1',
 'V00KSXLKN2',
 'V00KSXLKN3',
 'V00KSXLKN4',
 'V00KSXLKN5',
 'V00WOMSTFL',
 'V00WSLKN1',
 'V00WSLKN2',
 'V00WOMADLL',
 'V00DILKN1',
 'V00DILKN2',
 'V00DILKN3',
 'V00DILKN4',
 'V00DILKN5',
 'V00DILKN6',
 'V00DILKN7',
 'V00DILKN8',
 'V00DILKN9',
 'V00DILKN10',
 'V00DILKN11',
 'V00DILKN12',
 'V00DILKN13',
 'V00DILKN14',
 'V00DILKN15',
 'V00DILKN16',
 'V00DILKN17',
 'V00WOMTSL',
 'V00LKABPN',
 'V00LKLTTPN',
 'V00LKMTTPN',
 'V00LKRFXPN',
 'V00LKPATPN',
 'V00LKPGDPN',
 'V00LKPFCRE',
 'V00LKEFFB',
 'V00LKEFFPT',
 'V00LKFHDEG',
 'V00LKALNMT',
 'P01HPNL12',
 'P01HPL12CV',
 'P01HPNLIL',
 'P01HPNLOL',
 'P01HPNLFL',
 'P01HPNLB',
 'P01HPNLLB',
 'P01HPNLDK',
 'P01OJPNLA',
 

In [26]:
tmp = tmp.drop(columns=col_to_drop, axis=1)

In [27]:
labels_df_w_pain_names = tmp.copy()

In [28]:
pain_names.head()

,keys,category,specific meaning,comments
131,P02KPNRCV,knee symptoms,"right knee pain, aching or stiffness on most d...",NaN
132,P02KPNLCV,knee symptoms,"left knee pain, aching or stiffness on most da...",NaN
141,P01KPACDCV,knee symptoms,"days of limited activities due to knee pain, a...",NaN
142,P01KPA30CV,knee symptoms,avoided or changed activities to reduce knee p...,NaN
143,P01KPNREV,knee symptoms,"right knee pain, aching or stiffness: ever had...",NaN


In [29]:
pain_keys = pain_names['keys'].tolist()
pain_keys += [k+ '_C' for k in pain_keys if k in left['keys'].tolist()]  
pain_keys += [k+ '_I' for k in pain_keys if k in left['keys'].tolist()] 
pain_keys += [k+ '_C' for k in pain_keys if k in right['keys'].tolist()]  
pain_keys += [k+ '_I' for k in pain_keys if k in right['keys'].tolist()] 

In [30]:
len(pain_keys)

184

In [31]:
col_to_drop = [c for c in pain_keys if c in tmp.columns]

# labels_df_wo_pain_names = tmp.drop(columns=col_to_drop, axis=1)  

In [32]:
print(labels_df_w_pain_names.shape)

(4077, 994)


NameError: name 'labels_df_wo_pain_names' is not defined

In [251]:
labels_df_wo_pain_names.iloc[:5, :20]

,ID,side,dess_path,pain_meds,narcot,womac,koos,Inferred_KLG,oa_status,ID_side,pp_0,pp_1,pp_2,most_likely,set,sex,age_months,ethnicity,race,V00AGE
0,9477175.0,LEFT,9477175_LEFT_0.mat,0,0.0,0.0,100.0,1.0,0.0,9477175_LEFT,0.999999,5.458956e-07,6.750792e-08,0,2.0,F,864.0,Not Hispanic or Latino,White,72.0
1,9794339.0,LEFT,9794339_LEFT_0.mat,0,0.0,0.0,97.2,1.0,0.0,9794339_LEFT,0.999981,1.873607e-05,7.424682e-07,0,0.0,F,744.0,Not Hispanic or Latino,White,61.0
2,9282182.0,RIGHT,9282182_RIGHT_0.mat,0,0.0,0.0,100.0,0.0,0.0,9282182_RIGHT,0.999759,2.402764e-04,3.842015e-07,0,0.0,F,648.0,Not Hispanic or Latino,White,54.0
3,9359202.0,LEFT,9359202_LEFT_0.mat,0,0.0,0.0,100.0,3.0,1.0,9359202_LEFT,0.999816,1.823857e-04,1.635602e-06,0,1.0,M,828.0,Not Hispanic or Latino,White,69.0
4,9754338.0,RIGHT,9754338_RIGHT_0.mat,0,0.0,0.0,100.0,1.0,0.0,9754338_RIGHT,0.999820,1.789554e-04,8.681482e-07,0,2.0,F,744.0,Not Hispanic or Latino,White,62.0


## First prepare labels dataframe with pain variables

In [258]:
labels_df_w_pain_names = labels_df_w_pain_names.drop(['ID', 'side', 'ID_side', 'most_likely',
                                                      'pp_0', 'pp_1', 'pp_2',
                                                      'womac', 'koos', 'V00KOOSQOL', 'age_months'] , axis=1)

In [259]:
labels_df_w_pain_names = labels_df_w_pain_names.set_index('dess_path')

In [260]:
labels_df_w_pain_names = labels_df_w_pain_names.dropna(axis=1, thresh=3000)
labels_df_w_pain_names.shape

(4077, 369)

In [261]:
labels_df_w_pain_names.iloc[:5, :10]

,Unnamed: 0,pain_meds,narcot,Inferred_KLG,oa_status,set,sex,ethnicity,race,V00AGE
dess_path,,,,,,,,,,
9477175_LEFT_0.mat,0,0,0,1,0,2.0,F,Not Hispanic or Latino,White,72.0
9794339_LEFT_0.mat,1,0,0,1,0,0.0,F,Not Hispanic or Latino,White,61.0
9282182_RIGHT_0.mat,2,0,0,0,0,0.0,F,Not Hispanic or Latino,White,54.0
9359202_LEFT_0.mat,3,0,0,3,1,1.0,M,Not Hispanic or Latino,White,69.0
9754338_RIGHT_0.mat,4,0,0,1,0,2.0,F,Not Hispanic or Latino,White,62.0


# CHECKPOINT

In [4]:
# labels_df_w_pain_names.to_pickle('data/labels_df_w_pain_names_ncols_369.pickle')

In [5]:
labels_df_w_pain_names = pd.read_pickle('data/labels_df_w_pain_names_ncols_369.pickle')

In [6]:
# labels_df_w_pain_names = labels_df_w_pain_names.set_index('dess_path')
cat_names = []
cont_names = []
label_names = ['set', 'eta_0', 'eta_1', 'eta_2', 'eta_3','Unnamed: 0']

for n, t in zip(labels_df_w_pain_names.columns, labels_df_w_pain_names.dtypes):
    if t == 'O' and n not in label_names:
        cat_names.append(n)
    elif t != 'O' and n not in label_names:
        cont_names.append(n)
    else:
        print("Not assigning types: {}".format(n))

Not assigning types: set
Not assigning types: eta_0
Not assigning types: eta_1
Not assigning types: eta_2
Not assigning types: eta_3


In [7]:
cont_names

['V00AGE',
 'P01HEIGHT',
 'V00ABCIRC',
 'V00HT25MM',
 'V00WT25KG',
 'V00WTMAXKG',
 'V00WTMINKG',
 'P01KPACDCV',
 'V00KOOSFSR',
 'P01BPTOT',
 'P01BPBEDCV',
 'P01BPDAYCV',
 'V00BPSYS',
 'V00BPDIAS',
 'V00RPAVG',
 'V00CSPACE',
 'V00CSTIME1',
 'V00CSTIME2',
 'V0020MPACE',
 'V00STEPST1',
 'V00TIMET1',
 'V00STEPST2',
 'V00TIMET2',
 'V00HRB4WLK',
 'V00NUMSTOP',
 'V00400MTR',
 'V00400MTIM',
 'V00HR400WK',
 'V00PASE',
 'V00WKHR7CV',
 'V00HSPSS',
 'V00HSMSS',
 'V00COMORB',
 'V00CESD',
 'V00SMKPKYR',
 'V00PSMKYR',
 'V00DTACAR',
 'V00DTAIU',
 'V00DTANZN',
 'V00DTARE',
 'V00DTB1',
 'V00DTB12',
 'V00DTB6',
 'V00DTBCAR',
 'V00DTCAFFN',
 'V00DTCALC',
 'V00DTCARB',
 'V00DTCHOL',
 'V00DTCRYP',
 'V00DTCYST',
 'V00DTDAID',
 'V00DTDFIB',
 'V00DTFAT',
 'V00DTFE',
 'V00DTFOL',
 'V00DTGEN',
 'V00DTKCAL',
 'V00DTLIN',
 'V00DTLUT',
 'V00DTLYC',
 'V00DTMETH',
 'V00DTMG',
 'V00DTNA',
 'V00DTNIAC',
 'V00DTOLEC',
 'V00DTPHOS',
 'V00DTPOTA',
 'V00DTPROA',
 'V00DTPROT',
 'V00DTRET',
 'V00DTRIBO',
 'V00DTSFAT',
 'V00D

In [8]:
cat_names

['pain_meds',
 'narcot',
 'Inferred_KLG',
 'oa_status',
 'sex',
 'ethnicity',
 'race',
 'V00EDCV',
 'V00MARITST',
 'V00LIVENO',
 'V00INCOME',
 'V00CUREMP',
 'V00CEMPLOY',
 'V00HANDED',
 'P02JBMPCV',
 'P02FAMHXKR',
 'P02CNCR3',
 'P02IKPRISK',
 'P02KPN',
 'P02KINJ',
 'P02KSURG',
 'P02ACTRISK',
 'P02PA1',
 'P02PA2',
 'P02PA3',
 'P02PA4',
 'P01KPACT30',
 'P01KPA30CV',
 'P01SXKOA',
 'P01SVXRELK',
 'V00KOOSFX1',
 'V00KOOSFX4',
 'V00KOOSFX5',
 'V00KQOL1',
 'V00KQOL2',
 'V00KQOL3',
 'V00KQOL4',
 'V00KGLRS',
 'P01TMJE6M',
 'P01TJE30CV',
 'P01TJE30WC',
 'P01TMJF6M',
 'P01TJF30CV',
 'P01TJF30WC',
 'P01BP30',
 'P01BPACTCV',
 'P01RHBE',
 'P01RH1CV',
 'P01RH2CV',
 'P01RH3CV',
 'P01RH4CV',
 'P01RH5CV',
 'P01LHBE',
 'P01LH1CV',
 'P01LH2CV',
 'P01LH3CV',
 'P01LH4CV',
 'P01LH5CV',
 'V00KIKBALL',
 'V00CSTSGL',
 'V00CS5',
 'V00CSTREP1',
 'V00CSTREP2',
 'V00WLK20T1',
 'V00WLK20T2',
 'V00WLKAID',
 'V00WALKER',
 'V00HOSPSUR',
 'V00SAFEWLK',
 'V00CANEUSE',
 'V00400MCMP',
 'V00COMP10',
 'V00400EXCL',
 'V00DISC

In [9]:
with open('data/cont_cat_names_labels_df_w_pain_names.pickle', 'wb') as f:
    pickle.dump([cont_names, cat_names], f)

In [10]:
train_df = labels_df_w_pain_names.loc[labels_df_w_pain_names.set.isin([0, 1])].sort_values(by='set')

In [11]:
val_idx = [i for i, s in enumerate(train_df.set) if s == 1]

In [12]:
test_df = labels_df_w_pain_names.loc[labels_df_w_pain_names.set == 2]

In [13]:
test_tl = (TabularList.from_df(test_df, 
                        cat_names=cat_names,
                        cont_names=cont_names)
          )

In [19]:
test_tdb = (TabularList.from_df(test_df, 
                        cat_names=cat_names,
                        cont_names=cont_names,
                        procs=[FillMissing, Categorify, Normalize])
           .split_none()
           .label_from_df(['eta_0', 'eta_1', 'eta_2', 'eta_3'], label_cls=FloatList, log=False)
           .databunch()
          )

In [20]:
with open('data/test_tabular_tdb.pickle', 'wb') as f:
    pickle.dump(test_tdb, f)

In [17]:
tdb = (TabularList.from_df(train_df, 
                        cat_names=cat_names,
                        cont_names=cont_names,
                        procs=[FillMissing, Categorify, Normalize])
        .split_by_idx(val_idx)
        .label_from_df(['eta_0', 'eta_1', 'eta_2', 'eta_3'], label_cls=FloatList, log=False)
        .add_test(TabularList.from_df(test_df, 
                        cat_names=cat_names,
                        cont_names=cont_names))
        .databunch())

In [18]:
with open('data/train_tabular_tdb.pickle', 'wb') as f:
    pickle.dump(tdb, f)

In [173]:
tdb.show_batch(5)

pain_meds,narcot,Inferred_KLG,oa_status,sex,ethnicity,race,V00EDCV,V00MARITST,V00LIVENO,V00INCOME,V00CUREMP,V00CEMPLOY,V00HANDED,P02JBMPCV,P02FAMHXKR,P02CNCR3,P02IKPRISK,P02KPN,P02KINJ,P02KSURG,P02ACTRISK,P02PA1,P02PA2,P02PA3,P02PA4,P01KPACT30,P01KPA30CV,P01SXKOA,P01SVXRELK,V00KOOSFX1,V00KOOSFX4,V00KOOSFX5,V00KQOL1,V00KQOL2,V00KQOL3,V00KQOL4,V00KGLRS,P01TMJE6M,P01TJE30CV,P01TJE30WC,P01TMJF6M,P01TJF30CV,P01TJF30WC,P01BP30,P01BPACTCV,P01RHBE,P01RH1CV,P01RH2CV,P01RH3CV,P01RH4CV,P01RH5CV,P01LHBE,P01LH1CV,P01LH2CV,P01LH3CV,P01LH4CV,P01LH5CV,V00KIKBALL,V00CSTSGL,V00CS5,V00CSTREP1,V00CSTREP2,V00WLK20T1,V00WLK20T2,V00WLKAID,V00WALKER,V00HOSPSUR,V00SAFEWLK,V00CANEUSE,V00400MCMP,V00COMP10,V00400EXCL,V00DISCOMF,V00400PAIN,V00PASE1,V00PASE1HR,V00PASE2,V00PASE2HR,V00PASE3,V00PASE4,V00PASE5,V00PASE6,V00HOUACT1,V00HOUACT2,V00HOUACT3,V00HOUACT4,V00HOUACT5,V00HOUACT6,V00WORK7,V00PA130,V00PA130CV,V00PA130NM,V00PA230,V00PA230CV,V00PA330,V00PA330CV,V00PA430,V00PA430CV,V00PA530,V00PA530CV,V00SF1,V00SF2,V00SF3,V00SF4,V00SF5,V00SF6,V00SF7,V00SF8,V00SF9,V00SF10,V00SF11,V00SF12,V00HLTHCAR,V00HLTHCOV,V00MEDINS,P01RASTASV,P01RAIA,P01ARTHOTH,P01OADEGCV,P01OAHIPCV,P01OAHNDCV,P01OABCKCV,P01OAOTHCV,P01GOUTCV,P01OTARTCV,P01ARTDOC,P01ARTDRCV,P02KPMED,P02KPMEDCV,V00TYLEN,V00NSAIDS,V00NSAIDRX,V00COXIBS,V00NARCOT,V00SAME,V00MSM,V00DOXYCYC,V00PNMEDT,V00CHON,V00CHNFQCV,V00GLUC,V00GLCFQCV,V00KNINJ,V00HYINJCV,V00STINJCV,V00RXACTM,V00RXANALG,V00RXASPRN,V00RXBISPH,V00RXCHOND,V00RXCLCTN,V00RXCLCXB,V00RXCOX2,V00RXFLUOR,V00RXGLCSM,V00RXIHYAL,V00RXISTRD,V00RXMSM,V00RXNARC,V00RXNSAID,V00RXNTRAT,V00RXOSTRD,V00RXOTHAN,V00RXRALOX,V00RXRFCXB,V00RXSALIC,V00RXSAME,V00RXTPRTD,V00RXVIT_D,V00RXVLCXB,V00GNRH,V00PTH,V00BISPHOS,V00BISPTYP,V00HRTAT,V00HRTFAIL,V00BYPLEG,V00STROKE,V00ASTHMA,V00LUNG,V00ULCER,V00DIAB,V00KIDFXN,V00KIDTRAN,V00RA,V00POLYRH,V00LIVDAM,V00CANCER,V00CESD1,V00CESD2,V00CESD3,V00CESD4,V00CESD5,V00CESD6,V00CESD7,V00CESD8,V00CESD9,V00CESD10,V00CESD11,V00CESD12,V00CESD13,V00CESD14,V00CESD15,V00CESD16,V00CESD17,V00CESD18,V00CESD19,V00CESD20,V00FALL,V00FALLCV,V00BONEFX,V00SPNFX,V00SMOKE,V00SMOKER,V00PIPE,V00PSMOKER,V00DRNKAMT,V00DRKMORE,V00FFQFLG1,V00FFQFLG2,V00FFQFLG3,V00FFQFLG4,V00FFQFLG5,V00FFQ73,V00VIT1,V00VIT2,V00VIT3,V00VIT4,V00VIT5,V00VIT8,V00VIT9,V00VIT10,V00VIT11,V00VIT12,V00VIT6,V00VIT13,V00VIT7,P01HEIGHT_na,V00ABCIRC_na,V00HT25MM_na,V00WT25KG_na,V00WTMAXKG_na,V00WTMINKG_na,P01KPACDCV_na,V00KOOSFSR_na,P01BPTOT_na,P01BPBEDCV_na,P01BPDAYCV_na,V00RPAVG_na,V00CSPACE_na,V00CSTIME1_na,V00CSTIME2_na,V0020MPACE_na,V00STEPST1_na,V00TIMET1_na,V00STEPST2_na,V00TIMET2_na,V00HRB4WLK_na,V00NUMSTOP_na,V00400MTR_na,V00400MTIM_na,V00HR400WK_na,V00PASE_na,V00WKHR7CV_na,V00HSPSS_na,V00HSMSS_na,V00COMORB_na,V00CESD_na,V00SMKPKYR_na,V00PSMKYR_na,V00DTACAR_na,V00DTAIU_na,V00DTANZN_na,V00DTARE_na,V00DTB1_na,V00DTB12_na,V00DTB6_na,V00DTBCAR_na,V00DTCAFFN_na,V00DTCALC_na,V00DTCARB_na,V00DTCHOL_na,V00DTCRYP_na,V00DTCYST_na,V00DTDAID_na,V00DTDFIB_na,V00DTFAT_na,V00DTFE_na,V00DTFOL_na,V00DTGEN_na,V00DTKCAL_na,V00DTLIN_na,V00DTLUT_na,V00DTLYC_na,V00DTMETH_na,V00DTMG_na,V00DTNA_na,V00DTNIAC_na,V00DTOLEC_na,V00DTPHOS_na,V00DTPOTA_na,V00DTPROA_na,V00DTPROT_na,V00DTRET_na,V00DTRIBO_na,V00DTSFAT_na,V00DTVITC_na,V00DTVITD_na,V00DTVITE_na,V00DTVITK_na,V00DTZINC_na,V00DTSF_na,V00PCTALCH_na,V00PCTCARB_na,V00PCTFAT_na,V00PCTPROT_na,V00PCTSWT_na,V00BAPCARB_na,V00BAPFAT_na,V00BAPPROT_na,V00FIBBEAN_na,V00FIBGRN_na,V00FIBVGFR_na,V00SRVDRY_na,V00SRVFAT_na,V00SRVFRT_na,V00SRVGRN_na,V00SRVMEAT_na,V00SRVVEG_na,V00SUPB1_na,V00SUPB12_na,V00SUPB2_na,V00SUPB6_na,V00SUPBCAR_na,V00SUPCA_na,V00SUPCU_na,V00SUPFE_na,V00SUPFOL_na,V00SUPMG_na,V00SUPNIAC_na,V00SUPSE_na,V00SUPVITA_na,V00SUPVITC_na,V00SUPVITD_na,V00SUPVITE_na,V00SUPZINC_na,V00NERRORS_na,V00NFDSDAY_na,V00NNOSERV_na,V00NWARNS_na,V00PCTCOL1_na,V00PCTCOL9_na,V00PCTLARG_na,V00PCTMEDS_na,V00PCTSMAL_na,V00PCTXLS_na,V00AGE,P01HEIGHT,V00ABCIRC,V00HT25MM,V00WT25KG,V00WTMAXKG,V00WTMINKG,P01KPACDCV,V00KOOSFSR,P01BPTOT,P01BPBEDCV,P01BPDAYCV,V00BPSYS,V00BPDIAS,V00RPAV

In [174]:
with open('data/labels_df_w_pain_names_ncols_362.pickle', 'wb') as f:
    pickle.dump(tdb, f)

# Now labels dataframe that doesn't contain current pain-related var

In [34]:
labels_df_wo_pain_names = tmp.drop(['ID', 'side', 'ID_side', 'most_likely',
                                                      'pp_0', 'pp_1', 'pp_2',
                                                      'womac', 'koos', 'V00KOOSQOL', 'age_months'] , axis=1)

In [35]:
labels_df_wo_pain_names = labels_df_wo_pain_names.set_index('dess_path')

In [36]:
labels_df_wo_pain_names.iloc[:5, :10]

,pain_meds,narcot,Inferred_KLG,oa_status,set,sex,ethnicity,race,V00AGE,V00EDCV
dess_path,,,,,,,,,,
9477175_LEFT_0.mat,0,0.0,1.0,0.0,2.0,F,Not Hispanic or Latino,White,72.0,4: Some graduate school
9794339_LEFT_0.mat,0,0.0,1.0,0.0,0.0,F,Not Hispanic or Latino,White,61.0,5: Graduate degree
9282182_RIGHT_0.mat,0,0.0,0.0,0.0,0.0,F,Not Hispanic or Latino,White,54.0,3: College graduate
9359202_LEFT_0.mat,0,0.0,3.0,1.0,1.0,M,Not Hispanic or Latino,White,69.0,5: Graduate degree
9754338_RIGHT_0.mat,0,0.0,1.0,0.0,2.0,F,Not Hispanic or Latino,White,62.0,2: Some college


In [37]:
labels_df_wo_pain_names = labels_df_wo_pain_names.dropna(axis=1, thresh=3000)
labels_df_wo_pain_names.shape

(4077, 368)

In [38]:
labels_df_wo_pain_names.to_pickle('data/labels_df_wo_pain_names_ncols_368.pickle')

In [176]:
labels_df_wo_pain_names = pd.read_pickle('data/labels_df_wo_pain_names_ncols_368.pickle')

In [183]:
tmp = pd.read_pickle('data/labels_df_wo_pain_names_ncols_368.pickle')
tmp = tmp.sort_values(by='P01BMI', ascending=False)
tmp.iloc[40:60]

,pain_meds,narcot,Inferred_KLG,oa_status,set,sex,ethnicity,race,V00AGE,V00EDCV,...,V00VIT10,V00VIT11,V00VIT12,V00VIT6,V00VIT13,V00VIT7,eta_0,eta_1,eta_2,eta_3
dess_path,,,,,,,,,,,,,,,,,,,,,
9450507_RIGHT_0.mat,0,0,0,0,0.0,F,Not Hispanic or Latino,White,53.0,3: College graduate,...,0: No,0: No,0: No,0: No,0: No,0: No,9.234454,0.247862,-0.410830,0.337508
9878596_LEFT_0.mat,0,0,3,1,0.0,F,Not Hispanic or Latino,Black or African American,66.0,5: Graduate degree,...,0: No,0: No,0: No,0: No,0: No,0: No,2.071753,2.372712,-0.517709,1.225660
9878596_RIGHT_0.mat,0,0,3,1,0.0,F,Not Hispanic or Latino,Black or African American,66.0,5: Graduate degree,...,0: No,0: No,0: No,0: No,0: No,0: No,7.431667,-8.058194,-1.015473,-1.031921
9847829_RIGHT_0.mat,NaN,NaN,NaN,NaN,0.0,M,Not Hispanic or Latino,White,59.0,5: Graduate degree,...,0: No,0: No,0: No,0: No,0: No,0: No,24.902867,-3.027330,2.353961,0.564103
9355016_RIGHT_0.mat,0,0,0,0,0.0,F,Not Hispanic or Latino,White,58.0,5: Graduate degree,...,0: No,0: No,0: No,0: No,0: No,0: No,-4.731232,-0.203121,0.400390,-0.059551
9064631_LEFT_0.mat,0,0,1,0,1.0,F,Not Hispanic or Latino,White,54.0,2: Some college,...,0: No,0: No,0: No,1: Yes,0: No,0: No,7.117206,-3.486272,-0.530693,-0.261248
9355016_LEFT_0.mat,0,0,1,0,0.0,F,Not Hispanic or Latino,White,58.0,5: Graduate degree,...,0: No,0: No,0: No,0: No,0: No,0: No,5.091890,0.927082,3.584981,0.676560
9464409_RIGHT_0.mat,0,0,2,1,0.0,F,Not Hispanic or Latino,White,58.0,5: Graduate degree,...,0: No,0: No,0: No,1: Yes,1: Yes,0: No,-5.197373,-0.136162,0.023061,0.040672
9464409_LEFT_0.mat,0,0,2,1,0.0,F,Not Hispanic or Latino,White,58.0,5: Graduate degree,...,0: No,0: No,0: No,1: Yes,1: Yes,0: No,-3.853341,0.956026,-0.837154,-0.320773


In [188]:
tmp.loc['9990698_RIGHT_0.mat']

pain_meds                                         0
narcot                                            0
Inferred_KLG                                      0
oa_status                                         0
set                                               0
sex                                               F
ethnicity                    Not Hispanic or Latino
race                                          White
V00AGE                                           51
V00EDCV                          5: Graduate degree
V00MARITST                               1: Married
V00LIVENO                                    1: One
V00INCOME                        4: $50K to < $100K
V00CUREMP                                    1: Yes
V00CEMPLOY                         1: Works for pay
V00HANDED                           1: Right handed
P01BMI                                         16.9
P01HEIGHT                                      1643
P01WEIGHT                                      45.7
V00ABCIRC   

In [20]:
labels_df_wo_pain_names.iloc[:5, :15]

,pain_meds,narcot,Inferred_KLG,oa_status,set,sex,ethnicity,race,V00AGE,V00EDCV,V00MARITST,V00LIVENO,V00INCOME,V00CUREMP,V00CEMPLOY
dess_path,,,,,,,,,,,,,,,
9477175_LEFT_0.mat,0,0,1,0,2.0,F,Not Hispanic or Latino,White,72.0,4: Some graduate school,1: Married,1: One,5: $100K or greater,1: Yes,1: Works for pay
9794339_LEFT_0.mat,0,0,1,0,0.0,F,Not Hispanic or Latino,White,61.0,5: Graduate degree,1: Married,1: One,NaN,1: Yes,1: Works for pay
9282182_RIGHT_0.mat,0,0,0,0,0.0,F,Not Hispanic or Latino,White,54.0,3: College graduate,1: Married,2: Two,5: $100K or greater,1: Yes,1: Works for pay
9359202_LEFT_0.mat,0,0,3,1,1.0,M,Not Hispanic or Latino,White,69.0,5: Graduate degree,1: Married,1: One,4: $50K to < $100K,0: No,4: Not working other reasons
9754338_RIGHT_0.mat,0,0,1,0,2.0,F,Not Hispanic or Latino,White,62.0,2: Some college,1: Married,1: One,4: $50K to < $100K,1: Yes,1: Works for pay


In [21]:
labels_df_wo_pain_names.loc[:, ['narcot', 'Inferred_KLG', 'oa_status']] = labels_df_wo_pain_names.loc[:, 
                                                                          ['narcot', 'Inferred_KLG', 'oa_status']].astype('O')

In [22]:
labels_df_wo_pain_names.to_pickle('data/labels_df_wo_pain_names_ncols_368.pickle')

In [53]:
labels_df_wo_pain_names = pd.read_pickle('data/labels_df_wo_pain_names_ncols_368.pickle')

In [54]:
labels_df_wo_pain_names = labels_df_wo_pain_names.drop(['P01BMI', 'P01WEIGHT'], 1)

In [55]:
# labels_df_wo_pain_names = labels_df_wo_pain_names.set_index('dess_path')
cat_names = []
cont_names = []
label_names = ['set', 'eta_0', 'eta_1', 'eta_2', 'eta_3']

for n, t in zip(labels_df_wo_pain_names.columns, labels_df_wo_pain_names.dtypes):
    if t == 'O' and n not in label_names:
        cat_names.append(n)
    elif t != 'O' and n not in label_names:
        cont_names.append(n)
    else:
        print("Not assigning types: {}".format(n))

Not assigning types: set
Not assigning types: eta_0
Not assigning types: eta_1
Not assigning types: eta_2
Not assigning types: eta_3


In [56]:
cont_names

['V00AGE',
 'P01HEIGHT',
 'V00ABCIRC',
 'V00HT25MM',
 'V00WT25KG',
 'V00WTMAXKG',
 'V00WTMINKG',
 'P01KPACDCV',
 'V00KOOSFSR',
 'P01BPTOT',
 'P01BPBEDCV',
 'P01BPDAYCV',
 'V00BPSYS',
 'V00BPDIAS',
 'V00RPAVG',
 'V00CSPACE',
 'V00CSTIME1',
 'V00CSTIME2',
 'V0020MPACE',
 'V00STEPST1',
 'V00TIMET1',
 'V00STEPST2',
 'V00TIMET2',
 'V00HRB4WLK',
 'V00NUMSTOP',
 'V00400MTR',
 'V00400MTIM',
 'V00HR400WK',
 'V00PASE',
 'V00WKHR7CV',
 'V00HSPSS',
 'V00HSMSS',
 'V00COMORB',
 'V00CESD',
 'V00SMKPKYR',
 'V00PSMKYR',
 'V00DTACAR',
 'V00DTAIU',
 'V00DTANZN',
 'V00DTARE',
 'V00DTB1',
 'V00DTB12',
 'V00DTB6',
 'V00DTBCAR',
 'V00DTCAFFN',
 'V00DTCALC',
 'V00DTCARB',
 'V00DTCHOL',
 'V00DTCRYP',
 'V00DTCYST',
 'V00DTDAID',
 'V00DTDFIB',
 'V00DTFAT',
 'V00DTFE',
 'V00DTFOL',
 'V00DTGEN',
 'V00DTKCAL',
 'V00DTLIN',
 'V00DTLUT',
 'V00DTLYC',
 'V00DTMETH',
 'V00DTMG',
 'V00DTNA',
 'V00DTNIAC',
 'V00DTOLEC',
 'V00DTPHOS',
 'V00DTPOTA',
 'V00DTPROA',
 'V00DTPROT',
 'V00DTRET',
 'V00DTRIBO',
 'V00DTSFAT',
 'V00D

In [57]:
cat_names

['pain_meds',
 'narcot',
 'Inferred_KLG',
 'oa_status',
 'sex',
 'ethnicity',
 'race',
 'V00EDCV',
 'V00MARITST',
 'V00LIVENO',
 'V00INCOME',
 'V00CUREMP',
 'V00CEMPLOY',
 'V00HANDED',
 'P02JBMPCV',
 'P02FAMHXKR',
 'P02CNCR3',
 'P02IKPRISK',
 'P02KPN',
 'P02KINJ',
 'P02KSURG',
 'P02ACTRISK',
 'P02PA1',
 'P02PA2',
 'P02PA3',
 'P02PA4',
 'P01KPACT30',
 'P01KPA30CV',
 'P01SXKOA',
 'P01SVXRELK',
 'V00KOOSFX1',
 'V00KOOSFX4',
 'V00KOOSFX5',
 'V00KQOL1',
 'V00KQOL2',
 'V00KQOL3',
 'V00KQOL4',
 'V00KGLRS',
 'P01TMJE6M',
 'P01TJE30CV',
 'P01TJE30WC',
 'P01TMJF6M',
 'P01TJF30CV',
 'P01TJF30WC',
 'P01BP30',
 'P01BPACTCV',
 'P01RHBE',
 'P01RH1CV',
 'P01RH2CV',
 'P01RH3CV',
 'P01RH4CV',
 'P01RH5CV',
 'P01LHBE',
 'P01LH1CV',
 'P01LH2CV',
 'P01LH3CV',
 'P01LH4CV',
 'P01LH5CV',
 'V00KIKBALL',
 'V00CSTSGL',
 'V00CS5',
 'V00CSTREP1',
 'V00CSTREP2',
 'V00WLK20T1',
 'V00WLK20T2',
 'V00WLKAID',
 'V00WALKER',
 'V00HOSPSUR',
 'V00SAFEWLK',
 'V00CANEUSE',
 'V00400MCMP',
 'V00COMP10',
 'V00400EXCL',
 'V00DISC

In [58]:
with open('data/cont_cat_names_labels_df_wo_pain_names.pickle', 'wb') as f:
    pickle.dump([cont_names, cat_names], f)

In [59]:
train_df = labels_df_wo_pain_names.loc[labels_df_wo_pain_names.set.isin([0, 1])].sort_values(by='set')

In [60]:
val_idx = [i for i, s in enumerate(train_df.set) if s == 1]

In [61]:
test_df = labels_df_wo_pain_names.loc[labels_df_wo_pain_names.set == 2]

In [62]:
test_tl = (TabularList.from_df(test_df, 
                        cat_names=cat_names,
                        cont_names=cont_names)
          )

In [63]:
tdb = (TabularList.from_df(train_df, 
                        cat_names=cat_names,
                        cont_names=cont_names,
                        procs=[FillMissing, Categorify, Normalize])
        .split_by_idx(val_idx)
        .label_from_df(['eta_0', 'eta_1', 'eta_2', 'eta_3'], label_cls=FloatList, log=False)
        .add_test(TabularList.from_df(test_df, 
                        cat_names=cat_names,
                        cont_names=cont_names))
        .databunch())

In [64]:
tdb.show_batch(5)

pain_meds,narcot,Inferred_KLG,oa_status,sex,ethnicity,race,V00EDCV,V00MARITST,V00LIVENO,V00INCOME,V00CUREMP,V00CEMPLOY,V00HANDED,P02JBMPCV,P02FAMHXKR,P02CNCR3,P02IKPRISK,P02KPN,P02KINJ,P02KSURG,P02ACTRISK,P02PA1,P02PA2,P02PA3,P02PA4,P01KPACT30,P01KPA30CV,P01SXKOA,P01SVXRELK,V00KOOSFX1,V00KOOSFX4,V00KOOSFX5,V00KQOL1,V00KQOL2,V00KQOL3,V00KQOL4,V00KGLRS,P01TMJE6M,P01TJE30CV,P01TJE30WC,P01TMJF6M,P01TJF30CV,P01TJF30WC,P01BP30,P01BPACTCV,P01RHBE,P01RH1CV,P01RH2CV,P01RH3CV,P01RH4CV,P01RH5CV,P01LHBE,P01LH1CV,P01LH2CV,P01LH3CV,P01LH4CV,P01LH5CV,V00KIKBALL,V00CSTSGL,V00CS5,V00CSTREP1,V00CSTREP2,V00WLK20T1,V00WLK20T2,V00WLKAID,V00WALKER,V00HOSPSUR,V00SAFEWLK,V00CANEUSE,V00400MCMP,V00COMP10,V00400EXCL,V00DISCOMF,V00400PAIN,V00PASE1,V00PASE1HR,V00PASE2,V00PASE2HR,V00PASE3,V00PASE4,V00PASE5,V00PASE6,V00HOUACT1,V00HOUACT2,V00HOUACT3,V00HOUACT4,V00HOUACT5,V00HOUACT6,V00WORK7,V00PA130,V00PA130CV,V00PA130NM,V00PA230,V00PA230CV,V00PA330,V00PA330CV,V00PA430,V00PA430CV,V00PA530,V00PA530CV,V00SF1,V00SF2,V00SF3,V00SF4,V00SF5,V00SF6,V00SF7,V00SF8,V00SF9,V00SF10,V00SF11,V00SF12,V00HLTHCAR,V00HLTHCOV,V00MEDINS,P01RASTASV,P01RAIA,P01ARTHOTH,P01OADEGCV,P01OAHIPCV,P01OAHNDCV,P01OABCKCV,P01OAOTHCV,P01GOUTCV,P01OTARTCV,P01ARTDOC,P01ARTDRCV,P02KPMED,P02KPMEDCV,V00TYLEN,V00NSAIDS,V00NSAIDRX,V00COXIBS,V00NARCOT,V00SAME,V00MSM,V00DOXYCYC,V00PNMEDT,V00CHON,V00CHNFQCV,V00GLUC,V00GLCFQCV,V00KNINJ,V00HYINJCV,V00STINJCV,V00RXACTM,V00RXANALG,V00RXASPRN,V00RXBISPH,V00RXCHOND,V00RXCLCTN,V00RXCLCXB,V00RXCOX2,V00RXFLUOR,V00RXGLCSM,V00RXIHYAL,V00RXISTRD,V00RXMSM,V00RXNARC,V00RXNSAID,V00RXNTRAT,V00RXOSTRD,V00RXOTHAN,V00RXRALOX,V00RXRFCXB,V00RXSALIC,V00RXSAME,V00RXTPRTD,V00RXVIT_D,V00RXVLCXB,V00GNRH,V00PTH,V00BISPHOS,V00BISPTYP,V00HRTAT,V00HRTFAIL,V00BYPLEG,V00STROKE,V00ASTHMA,V00LUNG,V00ULCER,V00DIAB,V00KIDFXN,V00KIDTRAN,V00RA,V00POLYRH,V00LIVDAM,V00CANCER,V00CESD1,V00CESD2,V00CESD3,V00CESD4,V00CESD5,V00CESD6,V00CESD7,V00CESD8,V00CESD9,V00CESD10,V00CESD11,V00CESD12,V00CESD13,V00CESD14,V00CESD15,V00CESD16,V00CESD17,V00CESD18,V00CESD19,V00CESD20,V00FALL,V00FALLCV,V00BONEFX,V00SPNFX,V00SMOKE,V00SMOKER,V00PIPE,V00PSMOKER,V00DRNKAMT,V00DRKMORE,V00FFQFLG1,V00FFQFLG2,V00FFQFLG3,V00FFQFLG4,V00FFQFLG5,V00FFQ73,V00VIT1,V00VIT2,V00VIT3,V00VIT4,V00VIT5,V00VIT8,V00VIT9,V00VIT10,V00VIT11,V00VIT12,V00VIT6,V00VIT13,V00VIT7,P01HEIGHT_na,V00ABCIRC_na,V00HT25MM_na,V00WT25KG_na,V00WTMAXKG_na,V00WTMINKG_na,P01KPACDCV_na,V00KOOSFSR_na,P01BPTOT_na,P01BPBEDCV_na,P01BPDAYCV_na,V00RPAVG_na,V00CSPACE_na,V00CSTIME1_na,V00CSTIME2_na,V0020MPACE_na,V00STEPST1_na,V00TIMET1_na,V00STEPST2_na,V00TIMET2_na,V00HRB4WLK_na,V00NUMSTOP_na,V00400MTR_na,V00400MTIM_na,V00HR400WK_na,V00PASE_na,V00WKHR7CV_na,V00HSPSS_na,V00HSMSS_na,V00COMORB_na,V00CESD_na,V00SMKPKYR_na,V00PSMKYR_na,V00DTACAR_na,V00DTAIU_na,V00DTANZN_na,V00DTARE_na,V00DTB1_na,V00DTB12_na,V00DTB6_na,V00DTBCAR_na,V00DTCAFFN_na,V00DTCALC_na,V00DTCARB_na,V00DTCHOL_na,V00DTCRYP_na,V00DTCYST_na,V00DTDAID_na,V00DTDFIB_na,V00DTFAT_na,V00DTFE_na,V00DTFOL_na,V00DTGEN_na,V00DTKCAL_na,V00DTLIN_na,V00DTLUT_na,V00DTLYC_na,V00DTMETH_na,V00DTMG_na,V00DTNA_na,V00DTNIAC_na,V00DTOLEC_na,V00DTPHOS_na,V00DTPOTA_na,V00DTPROA_na,V00DTPROT_na,V00DTRET_na,V00DTRIBO_na,V00DTSFAT_na,V00DTVITC_na,V00DTVITD_na,V00DTVITE_na,V00DTVITK_na,V00DTZINC_na,V00DTSF_na,V00PCTALCH_na,V00PCTCARB_na,V00PCTFAT_na,V00PCTPROT_na,V00PCTSWT_na,V00BAPCARB_na,V00BAPFAT_na,V00BAPPROT_na,V00FIBBEAN_na,V00FIBGRN_na,V00FIBVGFR_na,V00SRVDRY_na,V00SRVFAT_na,V00SRVFRT_na,V00SRVGRN_na,V00SRVMEAT_na,V00SRVVEG_na,V00SUPB1_na,V00SUPB12_na,V00SUPB2_na,V00SUPB6_na,V00SUPBCAR_na,V00SUPCA_na,V00SUPCU_na,V00SUPFE_na,V00SUPFOL_na,V00SUPMG_na,V00SUPNIAC_na,V00SUPSE_na,V00SUPVITA_na,V00SUPVITC_na,V00SUPVITD_na,V00SUPVITE_na,V00SUPZINC_na,V00NERRORS_na,V00NFDSDAY_na,V00NNOSERV_na,V00NWARNS_na,V00PCTCOL1_na,V00PCTCOL9_na,V00PCTLARG_na,V00PCTMEDS_na,V00PCTSMAL_na,V00PCTXLS_na,V00AGE,P01HEIGHT,V00ABCIRC,V00HT25MM,V00WT25KG,V00WTMAXKG,V00WTMINKG,P01KPACDCV,V00KOOSFSR,P01BPTOT,P01BPBEDCV,P01BPDAYCV,V00BPSYS,V00BPDIAS,V00RPAV

In [65]:
with open('data/labels_df_wo_pain_names_ncols_365.pickle', 'wb') as f:
    pickle.dump(tdb, f)

In [122]:
# tmp = labels_df.loc[cat_names[0]].apply(lambda x:x.cat.codes)

In [6]:
def preprocess(labels_df, cat_names, cont_names):
    # preprocess categorical variables

    c = Categorify(cat_names, cont_names)
    c(labels_df)

    labels_df[cat_names] = labels_df[cat_names].apply(lambda x: x.cat.codes)
    
    
    # preprocess continuous vaiables
    f = FillMissing(cat_names, cont_names)
    n = Normalize(cat_names, cont_names)
    f(labels_df)
    n(labels_df)
    
    for t, cname in zip(labels_df.dtypes, labels_df.columns):
        if t == bool:
            labels_df[cname] = labels_df[cname].astype('int8')
    
    return labels_df

In [7]:
labels_df = preprocess(labels_df, cat_names, cont_names)

## Let's make dataset and dataloader

In [8]:
labels_df = labels_df.reset_index()

In [21]:
train_set = TabularData(labels_df, 'train')
val_set = TabularData(labels_df, 'val')

In [22]:
trainset_loader = DataLoader(train_set, 
                             batch_size=32,
                             shuffle=True,
                             num_workers=1,
                             drop_last=True)

valset_loader = DataLoader(val_set, 
                             batch_size=32,
                             shuffle=False,
                             num_workers=1,
                             drop_last=True)

In [27]:
from fastai.tabular import data

In [47]:
def def_emb_sz(classes, n, sz_dict={}):
    "Pick an embedding size for `n` depending on `classes` if not given in `sz_dict`."
    n_cat = len(classes[n].unique())
    sz = sz_dict.get(n, int(data.emb_sz_rule(n_cat)))  # rule of thumb
    return n_cat,sz

In [53]:
emb_sz = [def_emb_sz(train_set.labels_df, n) for n in cat_names]

In [54]:
model = TabularModel(emb_sz, len(cont_names), 
                     out_sz=3, 
                     layers=[256,128], 
                     ps=None, 
                     emb_drop=0.,
                     y_range=torch.tensor([-0.1, 1.1], device=defaults.device), 
                     use_bn=True)

In [63]:
learn = Learner(tdb, 
               model, 
               metrics=[mean_absolute_error])

In [ ]:
tdb.

In [64]:
learn.model.

LR Finder is complete, type {learner_name}.recorder.plot() to see the graph.


TypeError: forward() missing 1 required positional argument: 'x_cont'

In [ ]:
def tabular_learner(data:DataBunch, layers:Collection[int], emb_szs:Dict[str,int]=None, metrics=None,
        ps:Collection[float]=None, emb_drop:float=0., y_range:OptRange=None, use_bn:bool=True, **learn_kwargs):
    "Get a `Learner` using `data`, with `metrics`, including a `TabularModel` created using the remaining params."
    emb_szs = data.get_emb_szs(ifnone(emb_szs, {}))
    model = TabularModel(emb_szs, len(data.cont_names), out_sz=data.c, layers=layers, ps=ps, emb_drop=emb_drop,
                         y_range=y_range, use_bn=use_bn)
    return Learner(data, model, metrics=metrics, **learn_kwargs)

In [13]:
xb, yb = tdb.one_batch()

In [59]:
data_dir = PosixPath('data/')
dep_vars = ['pp_0', 'pp_1', 'pp_2']

In [60]:
procs = [FillMissing, Categorify, Normalize]    

In [63]:
test_tdb[0][:5]

pain_meds         0
narcot            0
womac             0
koos            100
Inferred_KLG      1
Name: 9477175_LEFT_0.mat, dtype: object

In [89]:
ItemList??

In [85]:
train_df.head()

,pain_meds,narcot,womac,koos,Inferred_KLG,oa_status,pp_0,pp_1,pp_2,sex,...,V00KQOL2,V00KQOL3,V00KQOL4,V00KGLRS,P02KPNRCV,P02KPNLCV,P01KPR30CV,P01KPL30CV,P01KPACTCV,P01HPR12CV
dess_path,,,,,,,,,,,,,,,,,,,,,
9794339_LEFT_0.mat,0,0,0.0,97.2,1,0,0.999981,1.873607e-05,7.424682e-07,F,...,0: Not at all,1: Mildly,1: Mild,0: Very good,0: No,1: Yes,0: No,0: No,0: No Limits or avoidance,0: No
9282182_RIGHT_0.mat,0,0,0.0,100.0,0,0,0.999759,2.402764e-04,3.842015e-07,F,...,0: Not at all,0: Not at all,1: Mild,0: Very good,0: No,0: No,0: No,1: Yes,0: No Limits or avoidance,0: No
9359202_LEFT_0.mat,0,0,0.0,100.0,3,1,0.999816,1.823857e-04,1.635602e-06,M,...,0: Not at all,0: Not at all,0: None,0: Very good,0: No,0: No,0: No,0: No,0: No Limits or avoidance,0: No
9371094_LEFT_0.mat,0,0,0.0,100.0,0,0,0.999998,5.268256e-07,1.587867e-06,M,...,0: Not at all,0: Not at all,0: None,0: Very good,0: No,0: No,0: No,0: No,0: No Limits or avoidance,0: No
9362978_LEFT_0.mat,1,0,0.0,97.2,2,1,0.875740,1.242253e-01,3.452085e-05,F,...,1: Mildly,0: Not at all,1: Mild,2: 2,0: No,0: No,0: No,0: No,1: Avoids,0: No


In [88]:
tmp = ((TabularList.from_df(train_df,
                           path=data_dir,
                           cat_names=cat_names,
                           cont_names=cont_names,
                           procs=procs))
        .split_by_idx(list(range(2665, len(train_df))))
        ._label_from_list(labels = train_df.loc[:, ['pp_0', 'pp_1']].values)
        .add_test(test_tdb)
        .databunch()
      )

KeyError: 'pp_0'

In [83]:
tmp2 = tmp._label_from_list(labels = train_df.loc[:, ['pp_0', 'pp_1']])

Exception: Your data isn't split, if you don't want a validation set, please use `split_none`.

In [27]:
small_data = ((TabularList.from_df(train_df,
                           path=data_dir,
                           cat_names=cat_names,
                           cont_names=cont_names,
                           procs=procs))
        .split_by_idx(list(range(2665, len(train_df))))
        .label_from_df(cols=dep_vars)
        .add_test(test_tdb)
        .databunch())

In [53]:
xb, yb = small_data.one_batch()